In [ ]:
from Metrics import fill_list, true_fill_list, cardinality_list, unique_values_list
from DataCleaning import df_cleaner, series_cleaner

import matplotlib as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sbn

In [ ]:
df = pd.read_json('data_analysis.json')

In [ ]:
df.head()
df.columns
df.count()
df.isnull().sum()

In [ ]:
#DATAFRAME unique values and features
unique = pd.DataFrame()
unique['features'] = df.columns
unique['unique_values'] = unique_values_list(df)
df2 = unique.sort_values('unique_values').reset_index(drop=True)
print df2
print df2.plot.bar(x=df2['features'])

In [ ]:
df['time_in_business'].unique()

In [ ]:
#Checking for non-unicode items (integers, null values, etc)
for i, column in enumerate(df.columns):
    print i, column
    for item in df[column].unique():
        if type(item) is not unicode:
            print item
    print

In [ ]:
#DATAFRAME metrics
df_result = pd.DataFrame()
df_result['features'] = df.columns
df_result['old_cardinality'] = unique_values_list(df)
df_result['true_cardinality'] = cardinality_list(df)
df_result['diff_cardinality'] = df_result['old_cardinality'] - df_result['true_cardinality']
df_result['fill_data'] = fill_list(df)
df_result['fill_rate'] = (df_result['fill_data']/len(df))
df_result['true_fill_data'] = true_fill_list(df)
df_result['true_fill_rate'] = df_result['true_fill_data'] /len(df)
df_result['diff_fill_data'] = df_result['fill_data'] - df_result['true_fill_data']

pd.options.display.float_format = '{:.4f}'.format
df_result

In [ ]:
fill_rate = df_result[['features','fill_data', 'fill_rate']]
fill_rate

In [ ]:
#DATAFRAME and GRAPH true fill rate

true_fill_rate = df_result[['features','true_fill_data', 'true_fill_rate']]
print true_fill_rate

graph_rate = true_fill_rate.iloc[:,0:2].sort(columns='true_fill_data')
ax = graph_rate.plot(x=graph_rate['features'], kind='bar', title='True-Valued Fill Rate', sort_columns=True, legend=False)
ax.set_ylabel("Number of Good Data Records")

In [ ]:
cardinality = df_result[['features', 'true_cardinality']].sort_values('true_cardinality').reset_index(drop=True)
cardinality

In [ ]:
df_clean = df_cleaner(df)

In [ ]:
#PLOT revenue distribution for all clean data

df_revenue = df_clean['clean_revenue'].value_counts().sort_values(ascending=False)
df_revenue = df_revenue.reindex(index = [u'less than $500,000', u'$500,000 to $1 million', u'$1 to 2.5 million',
       u'$2.5 to 5 million', u'$5 to 10 million', u'$10 to 20 million',
       u'$20 to 50 million', u'$50 to 100 million', u'$100 to 500 million',
        u'over $500 million', u'over $1 billion'])
df_revenue.plot(kind='bar', title='Revenue')

In [ ]:
#PLOT headcount distribution for all clean data

headcount = df_clean['clean_headcount'].value_counts().sort_values(ascending=False)
headcount = headcount.reindex(index = [u'1 to 4', u'5 to 9', u'10 to 19', u'20 to 49', u'50 to 99',
       u'100 to 249', u'250 to 499', u'500 to 999', u'over 1,000'])
headcount.plot(kind='bar', title='Headcount')

In [ ]:
#PLOT time_in_business distribution for all clean data
time = df_clean['clean_time_in_business'].value_counts().sort_values(ascending=False)
print time
time.plot(kind='bar', title='Time in Business')

In [ ]:
df_clean[df_clean['clean_time_in_business']=='less than a year']

In [ ]:
#Analyzing Billionaires 
billion = df_clean[df_clean['clean_revenue']=='over $1 billion']
s = billion['clean_headcount'].value_counts()
s = s.reindex(index = [u'1 to 4', u'5 to 9', u'10 to 19', u'20 to 49', u'50 to 99',
       u'100 to 249', u'250 to 499', u'500 to 999', u'over 1,000'])
s.plot(kind='bar', title= "Headcount in Businesses with Over $1 billion Revenue")

In [ ]:
min_billion = billion[billion['clean_headcount']=='1 to 4']

In [ ]:
min_billion['clean_name'].value_counts().sort_values(ascending=False)

In [ ]:
min_billion[min_billion['clean_name'].str.contains("san francisco")]

In [ ]:
min_billion[min_billion['clean_name']=='volunteers for outdoor colorado']

In [ ]:
min_billion[min_billion['clean_name'].str.contains("elementary")]

In [ ]:
#EXTRA
revenue_dfs = {}
for revenue in df_clean['clean_revenue'].unique():
    df3 = df_clean['clean_state'][df_clean['clean_revenue']==revenue]
    
    
    revenue_dfs[revenue] = pd.DataFrame({'state':df3.value_counts().index.tolist(),
                    'values':df3.value_counts().tolist()})
    
    
df_final = reduce(lambda left,right: pd.merge(left,right,on='state'), revenue_dfs.values()[1:])
column_names = ['state'] + revenue_dfs.keys()[1:]
df_final.columns = column_names
df_final['sum'] = df_final.sum(axis=1)
df_final.loc[len(df_final)]= df_final.iloc[:,1:].sum(axis=0)
df_final


#Exploring rows with null in the "names" column
null_name = df[df['name'].isnull()]
for i in null_name['address']:
    print df[df['address']==i]
    print 
